Attempting code in Google AI Quantum Paper (arXiv:2010.07965v1)

Will start with simple Fermi-Hubbard Model of 4 qubits (representing 4 spin states, and thus 2 particles).

In [53]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from qiskit import (
    QuantumCircuit, 
    Aer, 
    IBMQ, 
    assemble, 
    transpile,
    execute,
)
import openfermion as of
from openfermion import (
    linalg as of_lg
)
from qiskit.test.mock import FakeAthens
import math
import scipy
# from scipy import (
#     linalg as sc_lg
# )
IBMQ.save_account("5743a5c607636e43b7431c0370ec874af29cbf102ad69bcfc87a0fbbd1cf560fd841e69aef0e96eb0e1b6e3a59c5c3d72d89623b46f0964099e0195708aa9c05")

configrc.store_credentials:WARNING:2022-03-18 01:07:38,937: Credentials already present. Set overwrite=True to overwrite.


In [54]:
def getInvDiag(in_matrix):
	a = np.matmul(in_matrix, np.transpose(in_matrix.conjugate()))
	a = scipy.linalg.fractional_matrix_power(a, 0.5)

	return np.linalg.matrix_power(a, -1)

def GivensRotations():
	nsites = 8
	#* Parameters for setting up the Gaussian potential.
	l_up = 4
	m_up = 4.5
	sigma_up = 1
	#* For spin-dependent potentials

	site_index = np.arange(1, nsites + 1)
	spin_up_ham = np.diag([-1.] * (nsites - 1), k=1) + np.diag([-1.] * (nsites - 1), k=-1)
	spin_up_ham += np.diag(-l_up * np.exp(-0.5 * (((site_index - m_up)**2)) / (sigma_up**2)))
	spin_up_ham_u = np.matmul(getInvDiag(spin_up_ham), spin_up_ham)

	spin_down_ham = np.diag([-1.] * (nsites - 1), k=1) + np.diag([-1.] * (nsites - 1), k=-1)
	spin_down_ham_u = np.matmul(getInvDiag(spin_down_ham), spin_down_ham)

	up_decomp, up_diag = of_lg.givens_decomposition_square(spin_up_ham_u)
	down_decomp, down_diag = of_lg.givens_decomposition_square(spin_down_ham_u)

	# print(np.diag(np.linalg.eigh(spin_up_ham)[0]))
	# for t in up_decomp:
	# 	print("set = {}".format(t))

	return (up_decomp, down_decomp)

In [55]:
def getRootiSwapHC(qc=QuantumCircuit(2)):
    qc.rxx(math.pi / 4, 0, 1)
    qc.ryy(math.pi / 4, 0, 1)
    return qc.to_instruction(label="√iSWAP†")
    
def getRootiSwap(qc=QuantumCircuit(2)):
    qc.z(0)
    qc.append(getRootiSwapHC(), [0, 1])
    qc.z(0)
    return qc.to_instruction(label="√iSWAP")

def getGGate(a:float, gate_label:str, qc=QuantumCircuit(2)):
    qc.append(getRootiSwap(), [0, 1])
    qc.rz(-a, 0)
    qc.rz(a, 1)
    qc.append(getRootiSwapHC(), [0, 1])
    return qc.to_instruction(label=gate_label)
    
def getKGate(a:float, qc=QuantumCircuit(2), label:str="θ"):
    qc.rz(-(math.pi / 4), 0)
    qc.rz((math.pi / 4), 1)
    qc.append(getRootiSwap(), [0, 1])
    qc.rz(a, 0)
    qc.rz(-a, 1)
    qc.append(getRootiSwapHC(), [0, 1])
    qc.rz((math.pi / 4), 0)
    qc.rz(-(math.pi / 4), 1)
    return qc.to_instruction(label="K(" + label + ")")

In [59]:
def PreInitPrep(qc:QuantumCircuit):
    for i in range(8, 16):
        qc.x(i)

def InitialPreparation(up, qc=QuantumCircuit(8), c=0):
    qc.x(c)
    qc.x(c + 1)

    qc.barrier()

    up_G, down_G = GivensRotations()
    use = up_G if up else down_G
    gate_angles = {}
    for i in use:
        gate_angles[str(i[0][0]) + str(i[0][1])] = i[0][2] if ((str(i[0][0]) + str(i[0][1])) not in list(gate_angles.keys())) else gate_angles[str(i[0][0]) + str(i[0][1])] + i[0][2]
        if (len(i) == 2):
            gate_angles[str(i[1][0]) + str(i[1][1])] = i[1][2] if ((str(i[1][0]) + str(i[1][1])) not in list(gate_angles.keys())) else gate_angles[str(i[1][0]) + str(i[1][1])] + i[1][2]
    
    # print(gate_angles)
    for k in gate_angles.keys():
        if (abs(gate_angles[k]) < 1e-8):
            gate_angles[k] = 0
        
    gate_angles = dict(sorted(gate_angles.items()))

    for k in gate_angles.keys():
        if not(gate_angles[k] == 0):
            qc.append(getGGate(gate_angles[k], gate_label="G↑" if c==0 else "G↓"), [int(k.split("")[0]), int(k.split("")[1])])

    # print(gate_angles)
    return qc.to_instruction(label="initial")
    
def TrotterStep(qc=QuantumCircuit(16), c=0):
    
    for i in range(0, 15, 2):
        qc.append(getKGate(a=-0.3), [i, i+1])
    
    qc.barrier()
        
    for i in range(0, 7, 2):
        qc.cp(0.9, control_qubit=i, target_qubit=i+c)

    qc.barrier()
    
    for i in range(1, 6, 2):
        qc.iswap(i, i + 1)
        
    for i in range(1 + c, 6 + c, 2):
        qc.iswap(i, i + 1)

    qc.barrier()
    
    for i in range(2, 7, 2):
        qc.cp(0.9, control_qubit=i, target_qubit=i+c)
    
    qc.cp(0.9, control_qubit=7, target_qubit=15)
    
    qc.barrier()
    
    for i in range(1, 6, 2):
        qc.append(getKGate(a=(-0.3 + math.pi / 2), qc=QuantumCircuit(2), label="θ + π/2"), [i, i+1])
        
    for i in range(1 + c, 6 + c, 2):
        qc.append(getKGate(a=(-0.3 + math.pi / 2), qc=QuantumCircuit(2), label="θ + π/2"), [i, i+1])
    
    return qc.to_instruction(label="trotter")
        
new_model = QuantumCircuit(16)
PreInitPrep(new_model)
new_model.append(InitialPreparation(True), list(range(0, 8)))
new_model.append(InitialPreparation(False), list(range(8, 16)))
# new_model.append(TrotterStep(c=8), list(range(0, 16)))
# new_model.append(TrotterStep(c=8), list(range(0, 16)))
new_model.measure_all()

new_model.draw(output="mpl")

{'67': -6.81010803305071e-13, '56': 4.778399897986674e-13, '45': 2.786659791809143e-13, '34': 3.529366227907299, '23': 0.44185116427292914, '12': 0.8672365047089623, '01': -1.3343961579730967}
{'01': -1.3343961579730967, '12': 0.8672365047089623, '23': 0.44185116427292914, '34': 3.529366227907299, '45': 0, '56': 0, '67': 0}


In [52]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
device = provider.get_backend("ibmq_qasm_simulator")

counts = execute(new_model, device, shots=8000).result().get_counts()
plot_histogram(counts)

IBMQBackendApiError: "Error submitting job: '403 Client Error: Forbidden for url: https://api.quantum-computing.ibm.com/api/Network/ibm-q/Groups/open/Projects/main/Jobs/6233132a934a92144b328540/jobDataUploaded. Your job is too long., Error code: 3248.'"

In [ ]:
device = provider.get_backend("ibmq_qasm_simulator")

counts = execute(circuit, device).result().get_counts()
plot_histogram(counts)